Using PowerSystems to calculate network matrices

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSystems.jl supports the calculation of several different matrix representations of
power system networks. This example demonstrates how to use PowerSystems.jl to calculate:
 - Y bus
 - Power transfer distribution factor (PTDF)
 - Line outage distribution  factor (LODF)

### Dependencies
Let's use a dataset from the [tabular data parsing example](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/2_PowerSystems_examples/parse_matpower.ipynb)

In [1]:
using SIIPExamples

pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "2_PowerSystems_examples", "02_parse_matpower.jl"))

[ Info: extending matpower format with data: areas 1x3
[ Info: extending matpower format with data: gen_name 7x4
[ Info: extending matpower format by appending matrix "gen_name" in to "gen"
[ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
[ Info: the voltage setpoint on generator 4 does not match the value at bus 4
[ Info: the voltage setpoint on generator 1 does not match the value at bus 1
[ Info: the voltage setpoint on generator 5 does not match the value at bus 10
[ Info: the voltage setpoint on generator 2 does not match the value at bus 1
[ Info: the voltage setpoint on generator 3 does not match the value at bus 3
[ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
[ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
[ Info: removing 1 cost terms from generator 5: [1000.0, 0.0]
[ Info: removing 1 cost terms from generator 2: [1500.0, 0.0]
[ Info: removing 3 cost terms from generator 6: Float64[]
[ Info: removi

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


### Ybus

In [2]:
ybus = Ybus(sys)

[ Info: Validating connectivity with Goderya algorithm
[ Info: The System has no islands


PowerNetworkMatrix
:
  22.2507-222.484im  -3.52348+35.2348im  …  -15.4703+154.703im
 -3.52348+35.2348im   12.6911-126.898im              ⋅    
          ⋅          -9.16758+91.6758im              ⋅    
  -3.2569+32.569im            ⋅             -3.33367+33.3367im
 -15.4703+154.703im           ⋅               18.804-188.021im

### PTDF

In [3]:
ptdf = PTDF(sys)

PowerNetworkMatrix
:
  0.23245   -0.37175   -0.219627  0.0   0.19124
  0.41667    0.201807   0.119226  0.0   0.342801
  0.35088    0.169943   0.100401  0.0  -0.53404
  0.23245    0.62825   -0.219627  0.0   0.19124
  0.116225   0.314125   0.390186  0.0   0.0956199
  0.116225   0.314125   0.390186  0.0   0.0956199
 -0.35088   -0.169943  -0.100401  0.0  -0.46596

### LODF

In [4]:
lodf = LODF(sys)

PowerNetworkMatrix
:
 -1.0        0.398488   0.3581   -1.0       -0.360154  -0.360154  -0.3581
  0.542857  -1.0        0.6419    0.542857   0.195512   0.195512  -0.6419
  0.457143   0.601512  -1.0       0.457143   0.164642   0.164642   1.0
 -1.0        0.398488   0.3581   -1.0       -0.360154  -0.360154  -0.3581
 -0.5        0.199244   0.17905  -0.5       -1.0        0.639846  -0.17905
 -0.5        0.199244   0.17905  -0.5        0.639846  -1.0       -0.17905
 -0.457143  -0.601512   1.0      -0.457143  -0.164642  -0.164642  -1.0

### Indexing
Note that the axes of these matrices that correspond to buses are indexed by bus number
(::Int64) while the branch axes are indexed by branch name (::String). You can access
specific elements of the matrices as follows:

In [5]:
ptdf["bus3-bus4-i_6", 3]

0.39018648100730935

Additionally, PowerSystems provides accessors to the network matrices that take `Componets`
as arguments so that you can pass references to the components themselves rather than the
name or number. For example:

In [6]:
buses = collect(get_components(Bus, sys))
ybus[buses[1], buses[2]]

-3.2569046378322044 + 32.56904637832204im

If you would instead like to index by bus name, something like the following should work:

In [7]:
busname2num = get_components(Bus, sys) |> (c -> Dict(zip(get_name.(c), get_number.(c))))
ptdf["bus3-bus4-i_6", busname2num["bus3"]]

0.39018648100730935

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*